<a href="https://colab.research.google.com/github/sethkipsangmutuba/Python_Dashboard/blob/Deeplearning/a1vDashboard_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library Installation

In [ ]:
#!pip install pandas numpy faker dash dash-leaflet plotly

In [ ]:
#!pip install --upgrade pandas plotly

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#Dashboard

In [10]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime
import dash
from dash import dcc, html
import plotly.express as px
import dash_leaflet as dl

# Initialize Faker
fake = Faker()

# Define data
drug_categories = ['Antibiotics', 'Vaccines', 'Painkillers', 'Antivirals', 'Antifungals']
approval_statuses = ['Approved', 'Rejected', 'Pending']
years = pd.date_range(start='2019-01-01', end='2023-12-31', freq='M')

# Define specific locations within Kenya
kenya_locations = {
    'Nairobi': (-1.286389, 36.817223),
    'Mombasa': (-4.043477, 39.668206),
    'Kisumu': (-0.091702, 34.768159),
    'Eldoret': (0.514299, 35.269708),
    'Nakuru': (-0.303099, 36.066097),
    'Garissa': (-0.455635, 39.664963),
    'Kakamega': (0.283261, 34.751218),
    'Kitale': (1.002222, 35.014986),
    'Meru': (-0.049925, 37.652155),
    'Thika': (-1.033333, 37.083333)
}

# Generate synthetic data with realistic location coordinates
data = {
    'Date': [random.choice(years) for _ in range(500)],
    'Drug_Category': [random.choice(drug_categories) for _ in range(500)],
    'Approvals': [random.randint(50, 500) for _ in range(500)],
    'Approval_Status': [random.choice(approval_statuses) for _ in range(500)],
    'Adverse_Reactions': [random.randint(0, 50) for _ in range(500)],
    'Approval_Timeline': [random.randint(30, 180) for _ in range(500)],  # In days
    'Location': [random.choice(list(kenya_locations.keys())) for _ in range(500)]
}

# Add latitude and longitude columns based on the location
df = pd.DataFrame(data)
df['Latitude'] = df['Location'].map(lambda loc: kenya_locations[loc][0])
df['Longitude'] = df['Location'].map(lambda loc: kenya_locations[loc][1])

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a Dash layout
app.layout = html.Div([
    html.H1("Real-Time Dashboard with GIS Mapping"),
    dcc.Graph(
        id='approvals-trend',
        figure=px.line(df, x='Date', y='Approvals', title='Approvals Trend Over Time')
    ),
    dcc.Graph(
        id='category-distribution',
        figure=px.bar(df, x='Drug_Category', y='Approvals', color='Drug_Category', title='Drug Approvals by Category')
    ),
    dcc.Graph(
        id='status-pie-chart',
        figure=px.pie(df, names='Approval_Status', title='Approval Status Distribution')
    ),
    dcc.Graph(
        id='adverse-reactions-histogram',
        figure=px.histogram(df, x='Adverse_Reactions', title='Distribution of Adverse Reactions')
    ),
    html.Div([
        html.H3("Real-Time Location Map"),
        dl.Map([
            dl.TileLayer(),
            dl.LayerGroup([
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Popup(f'{loc}<br>Approvals: {df[(df["Latitude"] == lat) & (df["Longitude"] == lon)]["Approvals"].sum()}')
                    ]
                )
                for loc, lat, lon in zip(df['Location'], df['Latitude'], df['Longitude'])
            ])
        ], center=(-1.2921, 36.8219), zoom=6, style={'width': '1000px', 'height': '500px'})
    ])
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>